In [1]:
#flask dependencies
from flask import Flask , request
from werkzeug import secure_filename


import pandas as pd
import os
import json
from scipy.io import wavfile
import collections
from python_speech_features import mfcc
import csv
import wave
from scipy.io.wavfile import read
from sklearn import preprocessing as pp
import numpy as np
from sklearn.preprocessing import StandardScaler
import librosa
import librosa.display
import sklearn
import sklearn.ensemble
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import decomposition 
from sklearn import cross_validation
import pickle
import wave
from scipy.io.wavfile import read

C:\Users\Aditya Aggarwal\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
UPLOAD_FILE = '/Users/Aditya Aggarwal/Desktop/audio/'
UPLOAD_FOLDER = '/audio/'
ALLOWED_EXTENSIONS = set(['mp3' , 'wav'])

app = Flask(__name__)

app.config['UPLOAD_FILE'] = UPLOAD_FILE

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1] in ALLOWED_EXTENSIONS

@app.route('/find' , methods = ['GET' , 'POST'])
def find():
    if request.method == 'GET':
        return "HELLO"

@app.route('/' , methods = ['POST'])
def index():
    if request.method == 'POST':
        file = request.files['file']
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            print(filename)
            file.save(os.path.join(app.config['UPLOAD_FILE'], filename))
            
            filename = '/'+filename
            with open(r'C:\Users\Aditya Aggarwal\Desktop\audio'+filename , 'rb') as file:
                (sr , data) = read(file)
    
            d = data[: , 0]
            
            #mel-spectogram feature extraction.
            d_melspec = librosa.feature.melspectrogram(S = np.abs(librosa.stft(d.reshape(-1)))**2) 
            
            #mel-frequency cepstral coefficient feature extraction.-----> (13 features)
            d_mfcc = np.mean(librosa.feature.mfcc(np.array(d_melspec).flatten() , 44100 , n_mfcc = 13) , axis = 1)
            
            #extraction of delta and delta order 2. -----> (26 features)
            d_delta = librosa.feature.delta(d_mfcc)
            d_delta2 = librosa.feature.delta(d_mfcc , order = 2)
            
            #adding tempo feature to training data.------> (1 feature)
            d_oenv = librosa.onset.onset_strength(y = data[:,0], sr= 44100, hop_length = 512)
            d_tempo = librosa.beat.tempo(onset_envelope = d_oenv ,sr=44100,hop_length=512)[0]
            #in total 40 features by reducing (663552 features from raw music.)
            
            #adding all features to our dataset for prediction..
            dataset = [0]*40
            for j in range(13):
                dataset[j] = d_mfcc[j]
            for j in range(13):
                dataset[j+13] = d_delta[j]
            for j in range(13):
                dataset[j+26] = d_delta2[j]
            dataset[39] = d_tempo

            dataset = np.array(dataset)
        
        
            clf = pickle.load(open('random_forest.pkl' , 'rb'))
            ans = clf.predict(dataset.reshape(-1))
            music_class = str(ans[0]) 
            
            if(music_class == '1'):
                return "blues"
            elif(music_class == '2'):
                return "classical"
            elif(music_class == '3'):
                return "country"
            elif(music_class == '4'):
                return "disco"
            elif(music_class == '5'):
                return "hiphop"
            elif(music_class == '6'):
                return "jazz"
            elif(music_class == '7'):
                return "metal"
            elif(music_class == '8'):
                return "pop"
            else:
                return "error in predicting music!!!"
    
    
   

In [ ]:
if __name__ == '__main__':
    app.run(debug = True ,host="192.168.43.48", port = 5110,use_reloader=False)


 * Running on http://192.168.43.48:5110/ (Press CTRL+C to quit)
192.168.43.1 - - [30/Nov/2017 10:12:34] "GET /find HTTP/1.1" 200 -
192.168.43.1 - - [30/Nov/2017 10:12:35] "GET /favicon.ico HTTP/1.1" 404 -


sample.wav


C:\Users\Aditya Aggarwal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
192.168.43.1 - - [30/Nov/2017 10:13:02] "POST / HTTP/1.1" 200 -


sample.wav


C:\Users\Aditya Aggarwal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
192.168.43.1 - - [30/Nov/2017 10:29:11] "POST / HTTP/1.1" 200 -


sample.wav


C:\Users\Aditya Aggarwal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
192.168.43.1 - - [30/Nov/2017 10:29:36] "POST / HTTP/1.1" 200 -


In [1]:
import wave
from scipy.io.wavfile import read

with open(r'C:\Users\Aditya Aggarwal\Desktop\audio\blues.00002.wav' , 'rb') as file:
    (sr , data) = read(file)
    
d = data[: , 0]
print(d.shape)
print(sr)

    


(663552,)
44100


In [72]:
d_melspec = librosa.feature.melspectrogram(S = np.abs(librosa.stft(d.reshape(-1)))**2) 
#print(d_melspec)
#print(d_melspec.shape)
#print(librosa.feature.mfcc(np.array(d_melspec).flatten() , 44100 , n_mfcc = 13).shape)
d_mfcc = np.mean(librosa.feature.mfcc(np.array(d_melspec).flatten() , 44100 , n_mfcc = 13) , axis = 1)
#print(d_mfcc)
#print(d_mfcc.shape)
d_delta = librosa.feature.delta(d_mfcc)
#print('delta' ,d_delta)
#print(d_delta.shape)
d_delta2 = librosa.feature.delta(d_mfcc , order = 2)
#print(d_delta2)
#print(d_delta2.shape)
print(data[: , 0].shape)
d_oenv = librosa.onset.onset_strength(y = data[:,0], sr= 44100, hop_length = 512)
print(d_oenv)
print(d_oenv.shape)
d_tempo = librosa.beat.tempo(onset_envelope = d_oenv ,sr=44100,hop_length=512)[0]
print(d_tempo)
print(d_tempo.shape)



d_scale = pp.scale(d_delta2)



dataset = [0]*40
for j in range(13):
    dataset[j] = d_mfcc[j]
for j in range(13):
    dataset[j+13] = d_delta[j]
for j in range(13):
    dataset[j+26] = d_delta2[j]
dataset[39] = d_tempo

dataset = np.array(dataset)

print(dataset.shape)
print(dataset)





(663552,)
[ 0.          0.          0.         ...,  0.74742883  2.79828721
  2.56754588]
(1297,)
156.605113636
()
(40,)
[  2.19181928e+03   4.01879443e+01  -1.19526435e+01   8.82405000e+00
  -3.17195697e+00   5.75077720e+00  -3.85544662e-02   4.78466130e+00
  -7.55549553e-01   6.08807672e-01  -1.10400092e+00   1.95875777e+00
   2.06984057e+00  -3.64802098e+02  -3.64983502e+02  -3.29116387e+02
  -2.56396761e+02  -1.47595905e+02  -2.16117733e+00   3.76925704e-01
  -5.37637248e-01   5.47397557e-02  -2.39980330e-01   1.21497472e-01
   6.18799124e-02   3.69057083e-01  -9.11408122e+01  -8.20823028e+01
  -6.51584327e+01  -4.27720239e+01  -1.86003644e+01   1.77268754e+01
   4.23058342e+01   5.51720380e+01   5.77664526e+01   5.17696425e+01
   3.97057785e+01   2.45417830e+01   9.97193032e+00   1.56605114e+02]


In [69]:
sc = StandardScaler()
dataset1 = sc.fit_transform(dataset.transpose())
#dataset1 = dataset
print(sc)
print(dataset1.mean())
print(dataset1.shape)
print(dataset1)
pca = decomposition.PCA()
pca.fit_transform(dataset1)
print(dataset1.shape)
print(dataset1)


''' 0.38224914 -0.26196281 -0.3867268  -0.25350892 -0.65855687  0.4218158
  0.14770814  1.2952787  -0.84314272 -1.10821414 -1.42712105  0.32944424
  1.39275287 -0.38973252 -0.3845909  -0.38034262 -0.36914506 -0.35965402
  0.35819195  0.06993006 -0.05204642  0.47025967  0.48130802  1.02992736
  1.19481729  1.96136891 -0.38727615 -0.38632223 -0.3837401  -0.37488452
 -0.34465476  0.4281412   0.39794386  0.389768    0.38602376  0.38218642
  0.38173458  0.38595275  0.43537675  1.95255113'''




StandardScaler(copy=True, with_mean=True, with_std=True)
0.0
(40,)
[  5.95418735e+00   4.65099395e-02  -9.66510969e-02  -3.96050777e-02
  -7.25421976e-02  -4.80432817e-02  -6.39388970e-02  -5.06959207e-02
  -6.59075315e-02  -6.21614518e-02  -6.68642652e-02  -5.84549294e-02
  -5.81499323e-02  -1.06546054e+00  -1.06595861e+00  -9.67479222e-01
  -7.67814862e-01  -4.69083221e-01  -6.97669266e-02  -6.27981240e-02
  -6.53092154e-02  -6.36827415e-02  -6.44919466e-02  -6.34994466e-02
  -6.36631369e-02  -6.28197287e-02  -3.14075963e-01  -2.89204252e-01
  -2.42736829e-01  -1.81271057e-01  -1.14903569e-01  -1.51608249e-02
   5.23249739e-02   8.76513703e-02   9.47747860e-02   7.83095026e-02
   4.51860696e-02   3.55068708e-03  -3.64533730e-02   3.66153490e-01]
(40,)
[  5.95418735e+00   4.65099395e-02  -9.66510969e-02  -3.96050777e-02
  -7.25421976e-02  -4.80432817e-02  -6.39388970e-02  -5.06959207e-02
  -6.59075315e-02  -6.21614518e-02  -6.68642652e-02  -5.84549294e-02
  -5.81499323e-02  -1.0654605

C:\Users\Aditya Aggarwal\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\Aditya Aggarwal\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\Aditya Aggarwal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) i

' 0.38224914 -0.26196281 -0.3867268  -0.25350892 -0.65855687  0.4218158\n  0.14770814  1.2952787  -0.84314272 -1.10821414 -1.42712105  0.32944424\n  1.39275287 -0.38973252 -0.3845909  -0.38034262 -0.36914506 -0.35965402\n  0.35819195  0.06993006 -0.05204642  0.47025967  0.48130802  1.02992736\n  1.19481729  1.96136891 -0.38727615 -0.38632223 -0.3837401  -0.37488452\n -0.34465476  0.4281412   0.39794386  0.389768    0.38602376  0.38218642\n  0.38173458  0.38595275  0.43537675  1.95255113'

In [70]:

v = pca.explained_variance_
'''v = np.array([  2.04807938e+01 ,  5.25007010e+00 ,  3.66909133e+00  , 2.97473056e+00,
   2.16798598e+00 ,  1.33072274e+00  , 1.13282549e+00  , 9.08586232e-01,
   6.40566732e-01  , 5.22376300e-01 ,  3.60451932e-01 ,  2.76813458e-01,
   1.66724876e-01   ,1.18260456e-01 ,  4.11905892e-29 ,  1.61564975e-29,
   1.00640245e-29  , 7.29965419e-30 ,  5.78673713e-30 ,  3.98600590e-30,
   3.73802248e-30   ,2.94182267e-30 ,  2.62012193e-30 ,  2.34566610e-30,
   2.21117664e-30   ,1.87545533e-30 ,  1.72145710e-30 ,  1.54636617e-30,
   1.47028346e-30   ,1.24663708e-30 ,  1.02877988e-30 ,  8.00190949e-31,
   2.68244311e-31   ,1.92241514e-31 ,  1.92241514e-31 ,  1.92241514e-31,
   1.03422220e-31   ,8.31082587e-32  , 5.45000817e-32  , 4.95681470e-32])'''
print(v)
total = v.sum()
print(total)
sum = 0
i = 0
while i < v.shape[0] :
    sum += v[i]
    if (sum/total > 0.05) :
        break
    i +=1
pca.n_components = i
dataset1 = pca.fit_transform(dataset1)
print(dataset1.shape)
print(dataset1)

[ 0.]
0.0
(1, 1)
[[ 0.]]


C:\Users\Aditya Aggarwal\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\Aditya Aggarwal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\Aditya Aggarwal\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:398: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var


In [89]:
print(dataset.transpose().shape)
clf = pickle.load(open('GradientBoostingClassifier.pkl' , 'rb'))
ans = clf.predict(dataset.transpose())
print(ans)
clf

(40,)
[8]


C:\Users\Aditya Aggarwal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [91]:
print(dataset.reshape(-1).shape)
clf = pickle.load(open('DecisionTreeClassifier.pkl' , 'rb'))
ans = clf.predict(dataset.reshape(-1))
print(ans)
clf

(40,)
[7]


C:\Users\Aditya Aggarwal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [92]:
print(dataset.reshape(-1).shape)
clf = pickle.load(open('SVM.pkl' , 'rb'))
ans = clf.predict(dataset.reshape(-1))
print(ans)
clf

(40,)
[2]


C:\Users\Aditya Aggarwal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [93]:
print(dataset.reshape(-1).shape)
clf = pickle.load(open('KNeighbors.pkl' , 'rb'))
ans = clf.predict(dataset.reshape(-1))
print(ans)
clf

(40,)
[7]


C:\Users\Aditya Aggarwal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [94]:
print(dataset.reshape(-1).shape)
clf = pickle.load(open('random_forest.pkl' , 'rb'))
ans = clf.predict(dataset.reshape(-1))
print(ans)
clf

(40,)
[1]


C:\Users\Aditya Aggarwal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=8, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [105]:
a = [1]
a = str(a[0])
print(a)



1
